# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob, types
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch, Arc
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','MotorPreserved.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

speed vs max pos DEFINITION

In [ ]:
def plot_speed_pos_def(ax):
    step=.01
    x0=np.arange(-2,7,step)
    y0=10*x0+10

    x1=np.arange(7, 8.16,step)
    y1=-60*x1+500

    x=np.append(x0,x1)
    y=np.append(y0,y1)
    y=smooth(y*1.1,sigma=50,mode='constant')
    y=y[x>=0]
    x=x[x>=0]

    _x0=np.arange(-2,2.5,step)
    _y0=10*_x0+15

    _x1=np.arange(2.5, 9,step)
    _y1=-5*_x1+49.6

    _x=np.append(_x0,_x1)
    _y=np.append(_y0,_y1)
    _y=smooth(_y*1.25,sigma=50,mode='constant',cval=15)
    _y=_y[_x>=0]
    _x=_x[_x>=0]
    
    _x=_x[_y>y[-1]]
    _y=_y[_y>y[-1]]
    
    
    #plot the traj
    ax.plot(x, y, lw=2, c='k', )
    ax.plot(_x, _y, lw=2, c='k', )
    #plot the horizontal lines
    ax.axhspan(ymin=70, ymax=90, ec=None, fc='gray', alpha=.2)
    ax.axhspan(ymin=40, ymax=60, ec=None, fc='gray', alpha=.2)
    #plot the red piece
    x7=np.where(x>=7.3)[0][0]
    _x4=np.where(_x>=4)[0][0]
    LW=ax.spines['bottom'].get_lw()
    ax.plot(x[x7:x7+int(.3/step)],y[x7:x7+int(.3/step)],'r',lw=LW,zorder=5)
    ax.plot(_x[_x4:_x4+int(1.5/step)],_y[_x4:_x4+int(1.5/step)],'r',lw=LW,zorder=5)
    #plot the x-axis at the red piece
    ax.arrow(x[x7],y[x7],1.3,0,lw=LW,head_width=2,head_length=.2, ec='r', fc='r',zorder=5)
    ax.arrow(_x[_x4],_y[_x4],1.3,0,lw=LW,head_width=2,head_length=.2, ec='r', fc='r',zorder=5)
    # plot the curve
    ax.add_patch(Arc(xy=(x[x7],y[x7]), width=1, height=10, angle=0, theta1=271, theta2=360,
                     lw=LW, color='r', zorder=7))
    ax.add_patch(Arc(xy=(_x[_x4],_y[_x4]), width=1.5, height=15, angle=0, theta1=279, theta2=360,
                     lw=LW, color='r', zorder=7))
    #writing 
    ax.text(0,50,'Mid',rotation=0, c='k', fontsize='xx-small',ha='left',va='center')
    ax.text(0,80,'Back',rotation=0, c='k', fontsize='xx-small',ha='left',va='center')

    
    ax.set_xticks([0,1,2,3,4,5,6,7,8,9])
    ax.set_yticks(range(0,91,10))
    ax.set_yticklabels([0,'','','','','','','','',90])
    ax.spines['bottom'].set_bounds(0,9)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_bounds(0,90)
    ax.set_xlim([-.2,9])
    ax.set_ylim([-2,90])
    ax.set_ylabel('Position (cm)',fontsize='x-small',labelpad=-5)
    ax.set_xlabel('Trial time (s)',fontsize='x-small')
    ax.tick_params('both', labelsize='xx-small')

In [ ]:
if "__file__" not in dir():

    plt.close('all')
    fig=plt.figure(figsize=(2,2),dpi=600)
    ax=fig.add_subplot(111)
    plot_speed_pos_def(ax)

---

plot speed vs max pos trial-by-trial

In [ ]:
_MinX_=30
def maxPos_speed_correlation(data):
    _,maxTrials,maxPos=sequentialTrials(data, xFront=_MinX_)._compute_max_pos()
    spd=forwardRunningSpeed(data,minXBack=_MinX_+10,minXFront=_MinX_).compute()
    
    MaxTrialSpd={trial:spd[trial] for trial in maxTrials if trial in spd.keys()}
    MaxTrialPos=np.array([maxPos[i] for i,trial in enumerate(maxTrials) if trial in MaxTrialSpd.keys()])
    MaxTrialSpd=np.array(list(MaxTrialSpd.values()))
        
    return MaxTrialPos,MaxTrialSpd

def plot_animal_maxPos_speed_relation(root, ax, profile, colorCode, 
                                      Slice=slice(-5,None),animal_plot=False, goodAnimals=None):
    FrontBin=np.array([_MinX_+10,_MinX_+30])
    BackBin =np.array([70,90])
    MinNbTrials=10
    x_pos=[0,1]
    diff=x_pos[1]-x_pos[0]
    
    animalList=batch_get_animal_list(root, profile)
    if goodAnimals is not None:
        assert isinstance(goodAnimals, list)
        animalList=[animal for animal in animalList if animal in goodAnimals]
    
    pos={}
    spd={}
    for animal in animalList:
        data=data_fetch(root, animal, profile, PerfParam=[maxPos_speed_correlation], NbSession=Slice)['maxPos_speed_correlation']
        
        pos[animal]=np.array([i for sessionData in data for i in sessionData[0]])
        spd[animal]=np.array([i for sessionData in data for i in sessionData[1]])
    
    data=np.ones((len(spd),2))*np.nan
    for i,(animal,p) in enumerate(pos.items()):
        
        frontTrials=np.where(np.logical_and(p>=FrontBin[0], p<=FrontBin[1]))[0]
        if len(frontTrials) > MinNbTrials:
            data[i,0]=np.nanmean(spd[animal][frontTrials])

        backTrials=np.where(np.logical_and(p>=BackBin[0], p<=BackBin[1]))[0]
        if len(backTrials) > MinNbTrials:
            data[i,1]=np.nanmean(spd[animal][backTrials])


    y=np.nanpercentile(data,50,axis=0)
    yerr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(x_pos, y, abs(yerr-y),fmt='o', zorder=5, ms=2, elinewidth=1, color='k')
    ax.plot(x_pos, y, lw=1, color='k')
    
    #plotting individual animals
    if animal_plot:
        np.random.seed(seed=3)
        _coeff=50000
        for i in range(data.shape[0]):
            if np.any(np.isnan(data[i,:])):
                continue
            jitPre =np.random.uniform(low=x_pos[0]-diff/_coeff, high=x_pos[0]+diff/_coeff, size=1)
            jitPost=np.random.uniform(low=x_pos[1]-diff/_coeff, high=x_pos[1]+diff/_coeff, size=1)
            
            _,tag=lesion_type(root,animalList[i])
            c=colorCode[tag] if tag in colorCode else 'gray'
            ax.scatter([jitPre,jitPost],data[i,:], s=1, c=c, marker='o', edgecolors='none', alpha=.8)
            ax.plot([jitPre,jitPost],data[i,:], c=c, lw=.2, alpha=.4)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([x_pos[0]-diff/4,x_pos[-1]+diff/4])
    ax.set_xticks(x_pos)
    ax.spines['bottom'].set_bounds(*x_pos)
    ax.set_xticklabels(['Mid','Back'])
#     ax.set_xlabel('Max. Pos.')
    ax.xaxis.set_tick_params(rotation=0)

            
    return pos,spd,data

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
             }

    Profiles=(profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    preSlice=slice(-5,None)
    postSlice=slice(3,8)
    
    color={'DS':'r','DMS':'g', 'DLS':'b', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    a=plot_animal_maxPos_speed_relation(root, ax, profile1, color, Slice=slice(-3,None),animal_plot=True)

---

Free Exploration on immobile treadmill

In [ ]:
def _find_persistent_movement_epochs(pos: np.array, TrdSpeed: int, fs: int):
    """
    this function finds epochs in the position of a single trial, pos, where the animal has moved
    for a duration of at least, TH*cameraSamplingRate
    pos: position as a np.array
    """
    SpeedTolerance = 5 if TrdSpeed==0 else 10
    DurationTolerance= 0.4*fs
    
    pos_=pos#smooth(pos,.3*fs,mode="nearest")
    velocity=np.append(TrdSpeed-(np.diff(pos_)*fs),TrdSpeed)
    
    zeroVel=np.logical_and(velocity<SpeedTolerance, velocity>-SpeedTolerance)
    
    movement= np.logical_not(zeroVel)
    movement=movement*1

    dpcross=np.absolute( np.append(movement,0) - np.append(0,movement) )
    #any 1 in dpcross mean a zero-crossing in diff(pos)
    crossindex=np.where(dpcross)[0]
    crossindex=np.reshape(crossindex,(-1,2))
    movementepoch=crossindex[crossindex[:,1]-crossindex[:,0] > DurationTolerance]
    return movementepoch

def intertrial_displacement_loco_test(data,trial=None):
    if trial is None:
        trial=list(data.rawIntertrialPosition.keys())
    elif not hasattr(trial,'__iter__'):
        trial=[trial]
    
    dis=[0]
    for i in trial:
        assert data.dataType == 'ViewPoint'
        allTraj=smooth(data.rawIntertrialPosition[i],.3*data.cameraSamplingRateInterTrial)
        posMoveIndex=_find_persistent_movement_epochs(allTraj,
                                                      TrdSpeed=0,
                                                      fs=data.cameraSamplingRateInterTrial
                                                      )
        for i,j in posMoveIndex:
            epochPos=allTraj[int(i):int(j)]
            dis[-1]+=np.nansum(np.absolute(np.diff(epochPos)))
        dis.append(0)

    return np.array(dis[:-1])

In [ ]:
def plot_intertrial_displacement_loco(root,ax,sessionList,color, x_pos=1, **kwargs):
    sessionSlice=slice(None,10)

    dis=np.ones((sessionSlice.stop,len(sessionList)))*np.nan
    for i,session in enumerate(sessionList):
        data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
        dis[:,i]=intertrial_displacement_loco_test(data)[sessionSlice] / 100 #meters

    height=np.nanmedian (np.nansum(dis,axis=0))
    yerr  =np.nanstd    (np.nansum(dis,axis=0))

    ax.bar(x=x_pos, height=height, width=0.75, align='center',
          color='none', alpha=.9, zorder=1, **kwargs)

    np.random.seed(seed=12)
    sigma=0.2
    Color=[]
    for session in sessionList:
        _,tag=lesion_type(root,session[:6])
        Color.append(color[tag] if tag in color else 'k')

    jitter=np.random.uniform(low=x_pos-sigma, high=x_pos+sigma, size=len(sessionList))
    ax.scatter(jitter,np.nansum(dis,axis=0), s=4, c=Color, marker='o', edgecolors='none', alpha=.8, zorder=2)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xticks([x_pos])
    ax.set_ylabel(f'Locomotor activity\n (m/10min)',labelpad=0)
    ax.set_ylim([0,30])
    ax.set_xlim([ax.get_xlim()[0],x_pos+1])
    
    ax.tick_params(axis='x', direction='out')
    
    return dis

In [ ]:
if "__file__" not in dir():

    LesionLocoTest=  ['Rat325','Rat337','Rat341','Rat345','Rat365','Rat366',
                      'Rat372','Rat363','Rat362','Rat355','Rat354','Rat353']

    ControlLocoTest= ['Rat359','Rat360','Rat361','Rat356','Rat357','Rat358',
                      'Rat373','Rat349', 'Rat368','Rat367','Rat369','Rat371']

    profile={'Type':'Good',
             'rewardType':'Progressive',
             'Tag':['Reverse-Late_DLS-EXP','Early-Lesion_DMS-EXP',
                    'Early-Lesion_DS-EXP','Late-Lesion_DLS-EXP',
                    'Control-EXP','ImmobileTreadmill-EXP']
             }

    lesionSessionLoco  =batch_get_session_list(root,ControlLocoTest,profile)['Sessions']
    controlSessionLoco =batch_get_session_list(root,LesionLocoTest,profile)['Sessions']

    plt.close('all')
    ax=plt.figure().add_subplot(111);

    sessionList=lesionSessionLoco
    color='r'
    plot_intertrial_displacement_loco(root,ax,sessionList,color)
    sessionList=controlSessionLoco
    color='gray'
    plot_intertrial_displacement_loco(root,ax,sessionList,color,x_pos=2)

---

Forward Speed  for Loco Test 

In [ ]:
def _forwardRunningSpeed_loco(pos, cs: int, MinDur=1, MinDisplacement=30):
    MinDuration=MinDur*cs
    posLen=len(pos)
    posDiff=np.diff(a=pos, prepend=pos[0])
    ForwardDiff=(posDiff<=0).astype(int)
    edges=ForwardDiff - np.concatenate((np.array([False]),ForwardDiff[:-1]))
    try:
        ForwardEpoch=np.where(edges!=0)[0].reshape(-1,2)
    except ValueError:
        ForwardEpoch=np.where(edges!=0)[0][:-1].reshape(-1,2)

    _ForwardSpeed=[]
    for i,j in ForwardEpoch:
        if (j-i < MinDuration                   #short epoch
            or pos[i]-pos[j] < MinDisplacement  #short displacement
        ):
            continue

        _ForwardSpeed.append(((pos[i]-pos[j])/(j-i))*cs)
#         _ForwardSpeed.append(np.amax(np.diff(pos[i:j]))*cs)

    return _ForwardSpeed

def loco_forward_speed (data, trdSpeed):
    cs=int(data.cameraSamplingRate)
    speeds=[]
    
    for trial,pos in data.position.items():
        if data.treadmillSpeed[trial] != trdSpeed:
            continue
        speeds.extend(_forwardRunningSpeed_loco(pos=pos, cs=cs))
        
    return np.nanmean(speeds)

In [ ]:
_trdSpeed=np.array([0,10,15,20,25,30,35,40])
def plot_loco_forward_speed(root,ax,sessionList,color,offset=0):

    ForSpeed=np.ones((len(_trdSpeed),len(sessionList)))*np.nan
    for i,session in enumerate(sessionList):
        animal=session[:6]
        data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);

        for j,spd in enumerate(_trdSpeed):
            ForSpeed[j,i]=loco_forward_speed (data, spd)+spd
            
    ax.errorbar( _trdSpeed+offset, np.nanmedian(ForSpeed,axis=1), np.nanstd(ForSpeed,axis=1), color=color)
    ax.plot([_trdSpeed[0],_trdSpeed[-1]],[_trdSpeed[0],_trdSpeed[-1]],'k:',lw=1)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds(_trdSpeed[0],_trdSpeed[-1])
    ax.spines['left'].set_bounds(0,60)
    ax.set_xlim([_trdSpeed[0]-1,_trdSpeed[-1]+1])
    ax.set_xticks(_trdSpeed)
    ax.set_ylim([0,70])
    ax.set_ylabel('Running speed (cm/s)')
    ax.set_xlabel('Treadmill speed (cm/s)')
    
    return ForSpeed

In [ ]:
if "__file__" not in dir():

    LesionLocoTest=  ['Rat325','Rat337','Rat341','Rat345','Rat365','Rat366',
                      'Rat372','Rat363','Rat362','Rat355','Rat354','Rat353']

    ControlLocoTest= ['Rat359','Rat360','Rat361','Rat356','Rat357','Rat358',
                      'Rat373','Rat349', 'Rat368','Rat367','Rat369','Rat371']

    profile={'Type':'Good',
             'rewardType':'Progressive',
             'Tag':['Reverse-Late_DLS-EXP','Early-Lesion_DMS-EXP',
                    'Early-Lesion_DS-EXP','Late-Lesion_DLS-EXP',
                    'Control-EXP','ImmobileTreadmill-EXP']
             }

    lesionSessionLoco  =batch_get_session_list(root,ControlLocoTest,profile)['Sessions']
    controlSessionLoco =batch_get_session_list(root,LesionLocoTest,profile)['Sessions']

    plt.close('all')
    ax=plt.figure().add_subplot(111);

    sessionList=lesionSessionLoco
    color='r'
    plot_loco_forward_speed(root,ax,sessionList,color)
    sessionList=controlSessionLoco
    color='gray'
    plot_loco_forward_speed(root,ax,sessionList,color,offset=.2)

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='k'
    DLSColor='xkcd:orange'
    DMSColor='purple'
    DSColor='xkcd:green'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    BadLateRats=('Rat223','Rat231')
    colorSig='goldenrod'


    
    #===============================================
    
    # GRID 1 PARAMS
    
    profilePreLesion1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }

    profileLateLesions1={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': ['Late-Lesion_DLS',
                             'Late-Lesion_DMS','Late-Lesion_DMS-Sharp',
                             'Late-Lesion_DS','Late-Lesion_DS-Sharp']
                     }

    lesionSlice1 =slice(-5,None)
    ControlSlice1=slice(-5,None)

    
    
    
    #===============================================
    
    # GRID 6 PARAMs

    LesionLocoTest6=  ['Rat325','Rat337','Rat341','Rat345','Rat365','Rat366',
                      'Rat372','Rat363','Rat362','Rat355','Rat354','Rat353']

    ControlLocoTest6= ['Rat359','Rat360','Rat361','Rat356','Rat357','Rat358',
                      'Rat373','Rat349', 'Rat368','Rat367','Rat369','Rat371']

    profile6={'Type':'Good',
             'rewardType':'Progressive',
             'Tag':['Reverse-Late_DLS-EXP','Early-Lesion_DMS-EXP',
                    'Early-Lesion_DS-EXP','Late-Lesion_DLS-EXP',
                    'Control-EXP','ImmobileTreadmill-EXP']
             }

    lesionSessionLoco6  =batch_get_session_list(root,LesionLocoTest6,profile6)['Sessions']
    controlSessionLoco6 =batch_get_session_list(root,ControlLocoTest6,profile6)['Sessions']
    colorLesion6='gray'


Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(3.1,2.5)
    fig=plt.figure(figsize=figsize,dpi=600)



    ##########################################
    # 1: Speed MaxPos controlled
    gs1= fig.add_gridspec(nrows=1, ncols=2, left=0.4, bottom=0, right=1, top=.4, wspace=0.1)
    ax1l= fig.add_subplot(gs1[0])
    ax1r= fig.add_subplot(gs1[1])
    axes1=(ax1l,ax1r)
    
    *_,data1r=plot_animal_maxPos_speed_relation(root, ax1r, profileLateLesions1, ColorCode, 
                                               Slice=lesionSlice1,animal_plot=True)
    goodAnimalList1=batch_get_animal_list(root, profileLateLesions1)
    *_,data1l=plot_animal_maxPos_speed_relation(root, ax1l, profilePreLesion1, ColorCode,
                                               Slice=ControlSlice1,animal_plot=True,goodAnimals=goodAnimalList1)

    
    ax1l.text(0,0,f' $n={np.sum(np.sum(data1l,axis=1)>0)}$ rats',
              ha='left',va='bottom',fontsize='xx-small',transform=ax1l.transAxes)
    
    ax1r.text(0,0,f' $n={np.sum(np.sum(data1r,axis=1)>0)}$ rats',
              ha='left',va='bottom',fontsize='xx-small',transform=ax1r.transAxes)
    
    ax1l.set_title('Before',pad=0,fontsize='x-small',va='top')
    ax1r.set_title('Stable',pad=0,fontsize='x-small',va='top')
    for ax in axes1:
        ax.set_ylim([10,90])
        ax.spines['left'].set_bounds(10,90)
        ax.set_yticks(range(10,91,20))
        ax.set_ylabel('')
        
    ax1l.set_ylabel('Speed (cm/s)')
    ax1r.set_yticklabels('')
    
    totAx1=fig.add_subplot(gs1[:],frameon=False)
    totAx1.set_xlabel('Max. Pos. on treadmill')
    totAx1.yaxis.set_visible(False)
    totAx1.tick_params(color=(0, 0, 0, 0),labelcolor=(0, 0, 0, 0),zorder=-10)


    #STATS
    tmp=data1l[:,1]-data1l[:,0]
    tmp=tmp[~ np.isnan(tmp)]
    p1l,s1l=bootstrapTest(tmp)
    permtest.plotSigPair(ax1l,y=80,x=(0,1), s=s1l, color=colorSig,lw=.8)

    #STATS
    tmp=data1r[:,1]-data1r[:,0]
    tmp=tmp[~ np.isnan(tmp)]
    p1r,s1r=bootstrapTest(tmp)
    permtest.plotSigPair(ax1r,y=80,x=(0,1), s=s1r, color=colorSig,lw=.8)

    


    ##########################################
    # 1.5: MaxPos Speed Def
    gs1_5= fig.add_gridspec(nrows=1, ncols=1, left=-.05, bottom=0.05, right=.2, top=.35)
    ax1_5= fig.add_subplot(gs1_5[0])
    plot_speed_pos_def(ax1_5)

    

    ##########################################
    # 6: free exploration
    gs6= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=.6, right=.2, top=1)
    ax6= fig.add_subplot(gs6[0])

    dis6l=plot_intertrial_displacement_loco(root,ax6,controlSessionLoco6,color=ColorCode,x_pos=1,edgecolor=CtrlColor)
    dis6r=plot_intertrial_displacement_loco(root,ax6,lesionSessionLoco6,color=ColorCode,x_pos=2,edgecolor=colorLesion6)
    
    
#     ax6.text(1,1,f'$n={len(controlSessionLoco6)}$',
#               ha='center',va='bottom',fontsize='xx-small')
#     ax6.text(2,1,f'$n={len(lesionSessionLoco6)}$',
#               ha='center',va='bottom',fontsize='xx-small')

    
    ax6.set_xlim([0.38,2.62])
    ax6.set_xticks([1,2])
    ax6.set_xticklabels(['Control','Lesion'],rotation=45)
    
    for i,(ticklabel,tickline,c) in enumerate(zip(ax6.get_xticklabels(),
                                                  ax6.xaxis.get_ticklines()[::2],
                                                  [CtrlColor,colorLesion6]
                                                 )
                                             ):
        ticklabel.set_color(c)
        tickline.set_color(c)

    
    #STATS
    t6=permtest(np.median(dis6l,axis=0),np.median(dis6r,axis=0),nIterations=10000)
    s6='p={}'.format(SciNote(t6.pVal)) if t6.significantDiff[0] else 'n.s.'
    permtest.plotSigPair(ax6,y=28,x=(1,2), s=s6, color=colorSig,lw=.8)

    
    
    
    
    ##########################################
    # 7: free running 
    gs7= fig.add_gridspec(nrows=1, ncols=1, left=0.35, bottom=.6, right=.95, top=1)
    ax7= fig.add_subplot(gs7[0])
    
    spd7ctrl=plot_loco_forward_speed(root,ax7,controlSessionLoco6,color=CtrlColor,offset=0)
    spd7lesion=plot_loco_forward_speed(root,ax7,lesionSessionLoco6 ,color=colorLesion6,offset=0.6)

    
    #STATS
    t7=permtest(spd7ctrl.T,spd7lesion.T,nIterations=10000)
    t7.plotSignificant(ax=ax7,x=_trdSpeed,y=65,color=colorSig,lw=1.6)

    

    
#     fig.align_ylabels([ax1_5,ax6])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax6,ax7,ax1_5,axes1[0])
    OFFX=np.array([.07]*len(AXES))
    OFFY=np.array([.01]*len(AXES))
    OFFX=np.array([.07,.05,.07,.12,])
    OFFY=np.array([.05,.01,.05,.01,])
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','MotorPreserved.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()

In [ ]:
if "__file__" not in dir():

    a=data1r[:,1]/data1r[:,0]
    a=a[np.logical_not(np.isnan(a))]
    b=data1l[:,1]/data1l[:,0]
    b=b[np.logical_not(np.isnan(b))]
    test=permtest(a,b,nIterations=10000)
    print(test.pVal)